# Importing Libraries

In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from io import StringIO
import sqlite3

# Logs

In [18]:
def save_logs(message):
    with open('./input/logs/code_logs.txt','a') as f:
        f.write(f'{datetime.now()}: {message}')

In [19]:
datetime.now()

datetime.datetime(2025, 6, 21, 20, 2, 54, 611762)

# Extract


In [20]:
# URL of the webpage containing the table
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
# Send a GET request to the website
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table= soup.find('span',string='By market capitalization').find_next('table')
df_main= pd.read_html(StringIO(str(table)))[0]


In [21]:
def extract(url):
    print(url)
    soup = BeautifulSoup(requests.get(url).text,'html.parser')
    table= soup.find('span',string='By market capitalization').find_next('table')
    df =pd.read_html(StringIO(str(table)))[0]
    save_logs('Data has been extracted')
    return df
extract('https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks')

https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks


,Rank,Bank name,Market cap (US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [22]:
df_main

,Rank,Bank name,Market cap (US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


# Transform

In [23]:
def transform(df,csv_path):
    exchange_rate = pd.read_csv(csv_path,index_col=0).to_dict()['Rate']
    print(exchange_rate)
    df['Market cap (Pkr billion)s']= df['Market cap (US$ billion)']* exchange_rate['Pkr']
    df['Market cap (Inr billion)s']= df['Market cap (US$ billion)']* exchange_rate['Inr']
    df['Market cap (Eur billion)s']= df['Market cap (US$ billion)']* exchange_rate['Eur']
    df['Market cap (GBP billion)s']= df['Market cap (US$ billion)']* exchange_rate['GBP']
    #print(df)
    #save logs
    save_logs('Data transformation complete')
    
    return df


# Load

In [ ]:
database_name= './output/banks_db.db'
with sqlite3.connect(database_name) as conn:
    cursor = conn.cursor()
    result = conn.execute("select 1")
    print(result.fetchall())

[(1,)]


In [25]:
#load to sql lite
def load_data_sql(df,table_name,conn):
    cursor = conn.cursor()
    df.to_sql(table_name,conn,if_exists='replace',index=False)
    
def run_query(table_name,conn):
    cursor= conn.cursor()
    result=cursor.execute(('Select * from Largest_banks'))
    print(result.fetchall())    

In [26]:
def load_data(df,output_csv_path):
    df.to_csv(output_csv_path)
    
    save_logs('Data saved to csv file')
    return None

In [27]:
if __name__== '__main__':
    url= 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    csv_path='./input/exchange_rate.csv'
    output_csv_path = './output/transformed_exchange_rate.csv'
    database_name= './output/banks_db.db'
    table_name = 'Largest_banks'
    df =extract(url)
    df= transform(df,csv_path)
    load_data(df,output_csv_path)
    print(df)
    
    with sqlite3.connect(database_name) as conn:
        load_data_sql(df,table_name,conn)
        run_query(table_name,conn)

https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks
{'Pkr': 283.0, 'Inr': 86.0, 'Eur': 0.87, 'GBP': 0.74}
   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2                          Bank of America                    231.52   
2     3  Industrial and Commercial Bank of China                    194.56   
3     4               Agricultural Bank of China                    160.68   
4     5                                HDFC Bank                    157.91   
5     6                              Wells Fargo                    155.87   
6     7                        HSBC Holdings PLC                    148.90   
7     8                           Morgan Stanley                    140.83   
8     9                  China Construction Bank                    139.82   
9    10                            Bank of China                    136